In [6]:
import pandas as pd
import numpy as np

import networkx as nx

import pickle

In [7]:
df_place_details = pd.read_csv("../data/place_details.csv")
df_edges = pd.read_csv('../data/graph_edges_with_directionality - Kopie.csv')
G = nx.DiGraph()
for orig,desti in zip(df_edges['origin_place_id'], df_edges['destination_place_id']):
    G.add_edge(orig, desti)

In [8]:
df_place_details = pd.read_csv("../data/place_details.csv")
lat_long_dict = {pid: np.array([lat, long]) for pid, lat, long in zip(df_place_details['place_id'], df_place_details['lat'], df_place_details['lng'])}

In [9]:
nodes = list(G.nodes())

In [10]:
k = 0
path_lengths = {}
for start in nodes:
    for end in nodes:
        try:
            path = nx.shortest_path(G, start, end)
            path_length = 0
            for i in range(len(path)-1):
                path_length = path_length + np.sum( (lat_long_dict[path[i+1]] - lat_long_dict[path[i]])**2 )
        except nx.NetworkXNoPath:
            path_length = np.inf
        path_lengths[start, end] = path_length

In [11]:
lengths = np.array(list(path_lengths.values()))
max_length = max(lengths[lengths != np.inf])

In [12]:
start_end_dist = np.array([[start,end,l] for (start,end),l in path_lengths.items()])

In [14]:
np.savetxt('our_distances_intersection.csv', start_end_dist, delimiter = ',', fmt="%s", header = "start,end,cost", comments='')

In [15]:
df_distance = pd.read_csv('our_distances_intersection.csv', delimiter= ',')

In [18]:
nodes[:34] == list(np.array(list(path_lengths.keys()))[:34,1])

True

In [19]:
sensor_id_to_ind = {sid:i for i,sid in enumerate(nodes)}

In [20]:
dist_mx = np.zeros((len(nodes),len(nodes)), dtype=np.float32)

for i,start in enumerate(nodes):
    for j,end in enumerate(nodes):
        dist_mx[i,j] = path_lengths[start,end]

In [21]:
distances = dist_mx[~np.isinf(dist_mx)].flatten()
std = distances.std()
adj_mx = np.exp(-np.square(dist_mx / std))